# Scaling up

Originally released as part of a homework in ADA2019.

## Description

[Reddit](https://www.reddit.com/) aka *'the front page of the internet'* is a network of over a million *communities* aka *'subreddits'*, each of which covers a different topic based on people's interests. In other words, it is a *massive* collection of forums (corresponding to the aforementioned communities), where people can share content specific to a given topic or comment on other people’s posts.   

You are reddit's community manager and want to *appoint new moderators*. Because moderating a specific subreddit isn't a full-time job, you want the chosen moderators to moderate multiple subreddits at the same time. To make this choice effective, the moderators shouldn't have to spend too much time getting to know the community and the prevalent communication style, so it makes sense to let moderators moderate subreddits that are similar in communication style and language. At the same time, it also makes sense to let them moderate subreddits that are similar with respect to the participating users, because this allows moderators to track the behavior of individual users over multiple subreddits. For example, some users might only post offensive content once a month on a given subreddit, and therefore fly under the radar with someone moderating only that subreddit. However, considering all the subreddits these users post to, they might post something offensive every day but on different subreddits. Thus, a moderator in charge of all these subreddits would be able to ban such users much more effectively. In the light of the above description, your task is to find out ways to choose moderators considering both the textual content and the users of a subreddit.

### Dataset:
The dataset provided to you includes all the posts of the 15 largest subreddits written as of May 2015.

Reddit posts (provided to you via a [google drive folder](https://drive.google.com/a/epfl.ch/file/d/19SVHKbUTUPtC9HMmADJcAAIY1Xjq6WFv/view?usp=sharing))
```
reddit_posts
 |-- id: id of the post 
 |-- author: user name of the author 
 |-- body: text of the message
 |-- subreddit: name of the subreddit
```

Reddit scores (provided to you via a [google drive folder](https://drive.google.com/a/epfl.ch/file/d/1vr4PolJzTXr6ODSe3ucib5EAyp3rjxec/view?usp=sharing))
```
reddit_scores
 |-- id: id of the post 
 |-- score: score computed as sum of UP/DOWN votes
```

*Note: Jaccard similarity between subreddits represented using either the set of top-1000 words or the set of users can be computed locally (on the driver), however, all the other tasks have to be implemented in Spark.*

## B1. Getting a sense of the data

Start a PySpark instance...

In [2]:
import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *
import numpy as np 
import pandas as pd

conf = pyspark.SparkConf().setMaster("local[*]").setAll([
                                   ('spark.executor.memory', '12g'),  # find
                                   ('spark.driver.memory','4g'), # your
                                   ('spark.driver.maxResultSize', '2G') # setup
                                  ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

# FIX for Spark 2.x
locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

spark

... and load the data in a Spark dataframe.

In [3]:
reddit_posts = spark.read.json("messages.json.gz")

In [4]:
reddit_scores = spark.read.json("score.json.gz")

In [5]:
reddit_posts.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- id: string (nullable = true)
 |-- subreddit: string (nullable = true)



In [6]:
reddit_scores.printSchema()

root
 |-- id: string (nullable = true)
 |-- score: long (nullable = true)



### B1.1. Identify the most active subreddit

Print the list of subreddits along with the following information:
1. The total number of posts
2. The number of users with at least 1 message
3. The mean message length

*Note: Keep everything in one single dataframe and print the list sorted by number of posts in descending order.*

In [6]:
# post_count = reddit_posts.groupBy("subreddit")\
#                             .agg(count("*").alias("PostsCount"))\
#                             .sort(desc("PostsCount"))
# post_count = reddit_posts.groupBy("subreddit")\
#                             .agg(count("DISTINCT author").alias("Authors"),
#                                  count("*").alias("PostsCount"),
#                                  mean(len('body')).alias("MessageLength"))
subreddit_info = reddit_posts.groupBy("subreddit")\
     .agg(countDistinct("author").alias("users_count"),
          count("*").alias("total_posts"),
          avg(length("body")).alias("posts_length"))\
     .cache()
subreddit_info

DataFrame[subreddit: string, users_count: bigint, total_posts: bigint, posts_length: double]

In [7]:
subreddit_info.select("subreddit", "total_posts", "users_count", "posts_length").show(15)

+---------------+-----------+-----------+------------------+
|      subreddit|total_posts|users_count|      posts_length|
+---------------+-----------+-----------+------------------+
|         hockey|     389329|      25568| 95.37287230080472|
|         videos|     511492|     157628|170.22702603364274|
|          DotA2|     445154|      41466|141.48906670500546|
|           news|     477658|      98736| 230.9491602778557|
|  AdviceAnimals|     411902|     115815| 159.2513801826648|
| DestinyTheGame|     471160|      37008|165.41786866457255|
|           pics|     564502|     205305| 114.9710045314277|
|      worldnews|     439417|      99261|224.93754679495785|
|            nfl|     534345|      41593|148.96989211090212|
|         soccer|     455215|      41648|134.42224663071295|
|GlobalOffensive|     382017|      46686| 147.2883981602913|
|leagueoflegends|    1151287|     119321|152.72280760574904|
|         movies|     354601|      92484|164.83209297210104|
|            nba|     70

In [ ]:
reddit_posts.registerTempTable("posts")
query = """
SELECT subreddit, count(*) AS UsersCount
FROM 
    (SELECT subreddit, author
    FROM posts
    GROUP BY subreddit, author) AS users
GROUP BY subreddit
"""
# query = """
# SELECT subreddit, COUNT(DISTINCT author) AS UsersCount
# FROM posts
# GROUP BY subreddit
# """
users_count = spark.sql(query)
# users_count.show()

In [ ]:
# mean message length
len_body = reddit_posts.rdd.map(lambda post: (post.subreddit, (1, len(post.body))))
total_len = len_body.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
# mean_len = total_len.map(lambda subreddit: subreddit[1]/subreddit[0])
mean_len = total_len.map(lambda x: (x[0], x[1][1] / x[1][0]))
print(mean_len.collect())

### B1.2. Identify the largest subreddit

Print *two* different lists of subreddits: ordered by (1) the number of posts, and (2) the number of users. For each subreddit, print the name and the corresponding counts.

Additionally, (3) plot the mean of message length for each subreddit in descending order.

In [8]:
by_posts = subreddit_info.select("subreddit", "total_posts").sort(col("total_posts").desc()).toPandas()
display(by_posts)

,subreddit,total_posts
0,leagueoflegends,1151287
1,nba,704862
2,funny,691139
3,pics,564502
4,nfl,534345
5,videos,511492
6,news,477658
7,DestinyTheGame,471160
8,soccer,455215
9,DotA2,445154


### B1.3. Identify the subreddit with the highest average score

Print the list of subreddits sorted by their average content scores.

In [10]:
reddit_posts_id = reddit_posts.select("subreddit", "id")
post_with_scores = reddit_posts_id.join(reddit_scores, reddit_scores.id == reddit_posts_id.id)

# oblige d'avoir un alias
avg_scores = post_with_scores.groupBy("subreddit")\
    .agg(avg("score").alias("score"))\
    .sort(desc("score"))
    # .sort(col("score").desc())

avg_scores.show()

+---------------+------------------+
|      subreddit|             score|
+---------------+------------------+
|         videos|12.649445152612358|
|           pics|12.216559020162904|
|          funny|12.041505399058655|
|  AdviceAnimals|11.251695791717447|
|         soccer|10.634627593554693|
|         movies|  9.82014997137628|
|            nfl| 9.048348913155358|
|            nba| 9.032795071943161|
|           news| 8.673421150697779|
|      worldnews|  7.86683719564787|
|         hockey| 6.520120515039979|
|leagueoflegends| 5.983557531701479|
|          DotA2| 4.880537971129092|
|GlobalOffensive| 4.351442475073099|
| DestinyTheGame|3.0288819084811953|
+---------------+------------------+



## B2. Moderator assignment based on Subreddit Textual Content

Different subreddits follow different communication styles inherent in the topic and the community. Having said that, the goal is to discover similar subreddits by only looking at the *words* present in the posted messages. Once such a list of similar subreddits is identified, an appropriately chosen moderator can then be assigned to all these subreddits.

Specifically, the task boils down to computing a similarity score between two subreddits based on the *words* present in their textual content. Your first idea is to use the *Jaccard similarity*, which is defined as the size of the intersection of two sets divided by the size of their union.

$Jaccard(A,B) = \frac{|A \cap B|}{|A \cup B|}$

In [11]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

### B2.1.
The first step requires constructing a set representation of each subreddit. The goal is to represent each subreddit as a *set of words* existing in the messages posted on that subreddit. Compute the 50,000 most frequent words across all the provided subreddits. Construct a representation for each subreddit by retaining only the words found in the previously identified set of 50,000 frequent words.

Some rules:
 * Words are defined as tokens matching the regular expression `\W`
 * Remove all the stop-words (English language)

*Note: You might find the [RegexTokenizer](https://spark.apache.org/docs/2.2.0/ml-features.html#tokenizer) and the [StopWordsRemover](https://spark.apache.org/docs/2.2.0/ml-features.html#stopwordsremover) utilities available in the package pyspark.ml useful for this task as they help you in transforming the features and removing stopwords.*

In [12]:
# download tokenizer
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer

# tokenise the text MAP
# GROUPBY subreddit
# construct pairs of word, 1 MAP
# count each word occurence GROUPBY KEY
# sort in descending order SORT BY
# take 50 000 more frequent words LIMIT

# tokenize and clean the text
regexTokenizer = RegexTokenizer(inputCol="body", outputCol="all_words", pattern="\\W")
reddit_posts_with_all_words = regexTokenizer.transform(reddit_posts)

remover = StopWordsRemover(inputCol="all_words", outputCol="words")
reddit_posts_with_words = remover.transform(reddit_posts_with_all_words).drop("all_words")

# print(reddit_posts_with_words.take(1))
reddit_posts_with_words.show(1)

all_words = reddit_posts_with_words.select(explode("words").alias("word"))
top_50k = all_words.groupBy("word")\
    .agg(count("*").alias("word_count"))\
    .sort(col("word_count").desc())\
    .limit(50000)

top_50k.show()
# words = reddit_posts_with_words.select("subreddit", "words")\
#         .groupBy("subreddit")\
#         .rdd\
#         .map(lambda post: (post.subreddit, (token, 1)) for token in post.words_filt)\
#         .reduceByKey(lambda a, b: (a[0], a+b))\
#         .LIMIT(5000)

+---------+--------------------+-------+---------+--------------------+
|   author|                body|     id|subreddit|               words|
+---------+--------------------+-------+---------+--------------------+
|WyaOfWade|gg this one's ove...|cqug90h|      nba|[gg, one, watch, ...|
+---------+--------------------+-------+---------+--------------------+
only showing top 1 row

+------+----------+
|  word|word_count|
+------+----------+
|  like|   1018978|
|people|    745031|
|   get|    691349|
|   one|    682820|
| think|    575352|
|     m|    569057|
|    re|    517827|
|   com|    497303|
|  time|    456060|
|  good|    454768|
|  game|    452376|
|really|    451527|
|  even|    432973|
|  know|    413379|
|    gt|    404776|
|  http|    395552|
|  well|    352021|
|   see|    351081|
|  much|    348304|
|  also|    333478|
+------+----------+
only showing top 20 rows



In [13]:
# prendre le dataset avec les words
# filtrer les mots pas dans notre list
# groupby subreddit, en aggregate avec des union des listes de mots
# c'est bon c'est la set represntation!

# reddit_posts_with_words_top50 = reddit_posts_with_words.agg(lambda row: row)

# get_all_words in a subreddit
tokens = reddit_posts_with_words.select("subreddit", explode("words").alias("word")).distinct()
filtered_tokens = tokens.alias("t").join(top_50k, tokens.word == top_50k.word).select("t.*")
filtered_tokens

DataFrame[subreddit: string, word: string]

In [14]:
subreddit_50k = filtered_tokens.rdd\
    .map(lambda r: (r.subreddit, [r.word]))\
    .reduceByKey(lambda a, b: a+b)\
    .collect()

for sr in subreddit_50k:
    print("Subreddit: {} - Words: {}".format(sr[0], len(sr[1])))

### B2.2.
* Compute the Jaccard similarity between all the subreddits using the set representation obtained in step **B2.1.**, and plot in a heatmap the similarity values of all the pairs of subreddits.
* Analyze this plot and discuss your observations. Do you observe that subreddits corresponding to similar topics possess higher Jaccard similarity?
* Provide detailed interpretations of the obtained results. Specifically,
    - Explain the limitations of your conclusions, and discuss the potential reasons.
    - Explain the potential problems with the Jaccard similarity function.

In [ ]:
# sim = []
# for sr1 in subreddit_50k:
#     for sr2 in subreddit_50k:
#         sim_yo = jaccard_similarity(sr1.word, sr2.word)
#         sim.append(sim_yo)
#         print(f"{sr1.subreddit} and {sr2.subreddit} have similarity {sim_yo}")

# Note: similarity is computed 2 times! It can be optimized
similarity = []
for sr1 in subreddit_50k:
    for sr2 in subreddit_50k:
        similarity.append((sr1[0], sr2[0], jaccard_similarity(sr1[1], sr2[1])))

similarity_matrix_50k_words = pd.DataFrame(similarity).pivot(index=0, columns=1, values=2)
plot_heatmap(similarity_matrix_50k_words)

### B2.3.

* Alternatively, compute the 1000 most frequent words for each subreddit, construct its representation as the set of top-1000 words, and print a heatmap with the Jaccard similarity like in step **B2.2.**.
* Explain your observations in detail: how and why is this new result different from the one obtained in **B2.2.**?

*Note: Use the same rules specified in B2.1: words tokenized with the regex \W and stop-words removed*

In [ ]:
words_count_by_subreddit_rdd = reddit_posts_with_words.rdd\
    .flatMap(lambda r: [((r.subreddit, w), 1) for w in r.words])\
    .reduceByKey(lambda a,b: a+b).cache()

words_count_by_subreddit = spark.createDataFrame(
    words_count_by_subreddit_rdd.map(lambda r: Row(subreddit=r[0][0], word=r[0][1], count=r[1]))
)

window = Window.partitionBy(words_count_by_subreddit['subreddit'])\
    .orderBy(col('count').desc())

top1000_rdd = words_count_by_subreddit.select('*', rank().over(window).alias('rank'))\
    .filter(col('rank') <= 1000).rdd.map(lambda)
    

## B3. Moderator assignment based on Subreddit Users

Subreddits can be seen as communities of people interacting about a common topic. As an alternative to the *textual content* based similarity in **B2**, your task here is to validate if similarity between two subreddits can be measured based on their participating users.

Of course users are not monothematic, and they interact with multiple subreddits. In this task, we are specifically interested in observing the amount of overlap across different subreddits based on their participating users. Similar to **B2**, the overlap is measured using the *Jaccard similarity*.


### B3.1.
Construct a set representation of each subreddit as the users that posted at least one time in that subreddit.

Some users are very talkative and active across different topics. Print the username of the person that posted in the maximum number of subreddits. *Note that users who posted at least once in a subreddit are considered as participant of that subreddit.*

In [15]:
authors_by_subreddit_rdd = reddit_posts\
    .select("subreddit", "author").distinct()\
    .rdd\
    .reduceByKey(lambda a, b: a+b)\
    .map(lambda r: Row(author=r[0], commuinities=r[1]))
    # .map(lambda post: (post.subreddit, list(post.author)))\

authors_by_subreddit = spark.createDataFrame(authors_by_subreddit_rdd).sort(col("communities").desc())
authors_by_subreddit_rdd.take(1)

# df, puis rdd (map, reduceByKey), puis re df et sort

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 24) (10.10.197.215 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.api.python.PairwiseRDD.compute(PythonRDD.scala:130)
	at org.apache.spark.r

### B3.2.

* Compute the Jaccard similarity between all the subreddits using the set representation obtained in step **B3.1.**, and visualise it similar to **B2**.
* Analyze this plot, identify highly similar pairs of subreddits, and clearly describe your observations.

In [ ]:
users = reddit.select("subreddit", "author").distinct()\
    .rdd.map(lambda r: (r.subreddit, [r.author]))\
    .reduceByKey(lambda a,b: a+b).collect()

similarity_users = []
for sr1 in users:
    for sr2 in users:
        similarity_users.append((sr1[0], sr2[0], jaccard_similarity(sr1[1], sr2[1])))
        
similarity_matrix_users = pd.DataFrame(similarity_users).pivot(index=0, columns=1, values=2)

## B4. Language vs. Users similarity
    
* Visualize the similarity scores based on word (**B2.3.**) and user (**B3**) similarity on the x and y axes respectively for the subreddit `NBA` compared to all the other subreddits. Do some semantically meaningful groups emerge? Provide clear explanataions of your observations.
* Furthermore, do you observe differences in similarities between various sports-related subreddits in the dataset? Please provide explanations of the reasons behind these differences, if any!